## Natural Language Processing - Summer Term 2024
### Hochschule Karlsruhe
### Lecturer: Prof. Dr. Jannik Strötgen
### Tutor: Paul Löhr

# Exercise 04 brought to you bei Nele, Domi & Tobi

- Leftmost Derivation
- CKY Algorithm and Chomsky Normal Form
- Shift-Reduce Parsing

---

## Task 1 - Leftmost Derivation (pen and paper):

## Data and Grammar

Sentences:

 1. The angry bear chased the frightened little squirrel.
 2. The dog saw a man in the park.


Grammar:

- S -> NP VP
- NP -> Det Nom | Det N | NP PP
- Nom -> Adj Nom | Adj N
- VP -> V Adj | V NP | V S
- PP -> P NP
- Det -> ‘the’ | ‘a’
- N -> ‘bear’ | ‘squirrel’ | ‘dog’ | ‘park’ | ‘man’
- Adj -> ‘angry’ | ‘frightened’ | ‘little’
- V -> ‘saw’ | ‘chased’
- P -> ‘in’

Parse both sentences (1 and 2) with the Leftmost Derivation parser, using the grammar provided.

1. Write down each step and explain what you are doing in this step.

\# TEXT SUBMISSION ANSWER HERE 

### The angry bear chased the frightened little squirrel.

1. S -> NP VP
2. NP -> Det Nom
3. Det -> 'The' <br>
4. Nom -> Adj Nom <br>
    5a. Adj -> 'angry' <br>
    6a. Nom -> Adj Nom <br>
    7a. Adj -> ERROR <br>
4. Nom -> Adj N <br>
5. Adj -> 'angry'
6. N -> 'bear'
7. VP -> V NP
8. V -> 'chased'
9. NP -> Det Nom
10. Det -> 'the'
11. Nom -> Adj Nom
12. Adj -> 'frightened'
13. Nom -> Adj N
14. Adj -> 'little'
15. N -> 'squirrel'

```
         S                                                | 1
      ___|___
      |       |
     NP      VP                                           | 2
  ___|___     |________________                           | 3 
  |     |         |            |                             
 Det    Nom       |            |                          | 5
  |    __|__      |            |                          | 6 
  |    |    |     |            |                          | 7
  |   Adj   N     |            |                          | 8 
  |    |    |     |            |                          | 9
  |    |    |     |            |
  |    |    |     V            NP                         | 10   
  |    |    |     |      ______|______                    | 11
  |    |    |     |     |             |
  |    |    |     |    Det           Nom                  | 12
  |    |    |     |     |        _____|_____              | 13
  |    |    |     |     |       |           |
  |    |    |     |     |      Adj         Nom            | 14
  |    |    |     |     |       |         __|___          | 15
  |    |    |     |     |       |        |      |         | 16
  |    |    |     |     |       |       Adj     N         
  |    |    |     |     |       |        |      | 
 The angry bear chased the frightened little squirrel
```


### The dog saw a man in the park.
1. S -> NP VP
2. NP -> Det N
3. Det -> 'The'
4. N -> 'dog'
5. VP -> V NP
6. V -> 'saw'
7. NP -> NP PP
8. NP -> Det N
8. Det -> 'a'
9. N -> 'man'
10. PP -> P NP
11. P -> 'in'
12. NP -> Det N
13. Det -> 'the'
14. N -> 'park' 

In [1]:
import nltk

def build_tree(grammar, sentence):
    parser = nltk.LeftCornerChartParser(grammar)
    trees = parser.parse(sentence)
    for tree in trees:
        return tree

# Define the grammar
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det Nom | Det N | NP PP
    Nom -> Adj Nom | Adj N
    VP -> V Adj | V NP | V S
    PP -> P NP
    Det -> 'the' | 'a' | 'The' 
    N -> 'bear' | 'squirrel' | 'dog' | 'park' | 'man'
    Adj -> 'angry' | 'frightened' | 'little'
    V -> 'saw' | 'chased'
    P -> 'in'
""")
# Adjustment in Det needed, because function is case sensitive

# Define the sentence
sentence1 = "The angry bear chased the frightened little squirrel".split()
sentence2 = "The dog saw a man in the park".split()

# Build the tree
tree1 = build_tree(grammar, sentence1)
tree2 = build_tree(grammar, sentence2)

# Display the tree
print("Deriviation Tree 1:", tree1)
print("Deriviation Tree 2:", tree2)



Deriviation Tree 1: (S
  (NP (Det The) (Nom (Adj angry) (N bear)))
  (VP
    (V chased)
    (NP
      (Det the)
      (Nom (Adj frightened) (Nom (Adj little) (N squirrel))))))
Deriviation Tree 2: (S
  (NP (Det The) (N dog))
  (VP
    (V saw)
    (NP (NP (Det a) (N man)) (PP (P in) (NP (Det the) (N park))))))


2. Discuss the pros and cons of this approach.

- Pros: Easy to follow through
- Cons: Backtracking

---

## Task 2 - CKY Algorithm:

## Part 1: Theory

### Why is bottom-up parsing inefficient? 
Bottom-up parsing is inefficient, because wrong combinations can be made which lead to incorrect parsing trees. (not everything that can be combinded, should be combined)

If a parsing tree is correct can only be evaluated at the end. The Algorithm needs to start from the beginning if the parisng tree is wrong.
### How does CKY algorithm overcome this issue?
The CKY algortithm is overcoming this issue through the caching of intermediate results and backtracking.

## Part 2: Chomsky Normal Form (pen and paper)

**(1)** Why do we need to transform the grammar to Chomsky Normal Form 
before applying the CKY algorithm?

CNF ensures that production rules have specific forms required by the CKY algorithm
- Rules with three terminal symbols can't be applied, because in a table for CKY only two terminal symbols can be combined.
- Rules that only refer to one terminal symbol are also problematic since they cant be combined to the left side of the rule.


For Example:

Mary led John with care

```
-------------------------------------------
Mary| No? |  -   |  - | -      |    -     |
    ---------------------------------------
     led  | Verb |  - |  VP    |    -     |
           --------------------------------
            John | No |   -     |    -     |
                 --------------------------
                 with |  Prep  |    PP    |
                        -------------------
                         care  |    Adj   |
                            ---------------


**(2)** Transform the given example to Chomsky Normal Form.

- S -> NP VP
- NP -> Noun
- NP -> Noun PP
- VP -> Verb NP
- VP -> Verb NP PP
- PP -> Prep Adj
- Noun -> ‘John‘ | ‘Mary‘
- Verb -> ‘led‘
- Prep -> ‘with‘
- Adj -> ‘care‘

It already doesn't contain eplsilon-productions. 
First step is then to remove all unit productions (NP -> Noun).
Second we reomve all rules with more than two non-terminals (binarization) (VP -> Verb NP PP)

### Option A)

- S -> NP VP
- **NP** -> ‘John‘ | ‘Mary‘ | Noun PP
- **VP** -> **VP0** PP | Verb NP
- **VP0** -> Verb NP
- Noun -> ‘John‘ | ‘Mary‘
- PP -> Prep Adj
- Verb -> ‘led‘
- Prep -> ‘with‘
- Adj -> ‘care‘

### Option B)

- S -> NP VP
- **NP** -> 'John' | 'Mary' | Noun PP
- VP -> Verb NP | Verb **VP1**
- **VP1** -> NP PP
- PP -> Prep Adj
- Noun -> ‘John‘ | ‘Mary‘
- Verb -> ‘led‘
- Prep -> ‘with‘
- Adj -> ‘care‘

---

## Task 3 - CYK algorithm (pen and paper):

Parse the first sentence from Task 1, this time using the CYK algorithm:
    
    The angry bear chased the frightened little squirrel.
    
Again, use the initial grammar from Task 1 as a reference:

Grammar:

- S -> NP VP 
- NP -> Det Nom | Det N | NP PP
- Nom -> Adj Nom | Adj N
- VP -> V Adj | V NP | V S
- PP -> P NP
- Det -> ‘the’ | ‘a’
- N -> ‘bear’ | ‘squirrel’ | ‘dog’ | ‘park’ | ‘man’
- Adj -> ‘angry’ | ‘frightened’ | ‘little’
- V -> ‘saw’ | ‘chased’
- P -> ‘in’

The grammar needs to be transfered to CNF as in Task 2. This is the case here since there are no epsilon-productions, no unit productions and no rules with more than two non-terminals 


#### Sentence 1

The angry bear chased the frightened little squirrel

```
---------------------------------------------------------------------
The |  Det |  -    |  NP  | -   |    -     |   -   |    -    |  S   |
    -----------------------------------------------------------------
     angry |  Adj  |  Nom |  -  |    -     |   -   |    -    |  -   |
           ----------------------------------------------------------
              bear |   N  |  -  |    -     |   -   |    -    |  -   |
                 ----------------------------------------------------
                   chased |  V  |    -     |   -   |    -    |  VP  |
                        ---------------------------------------------
                            the |   Det    |   -   |    -    |  NP  |
                            -----------------------------------------
                               frightened  |  Adj  |    -    |  Nom |
                                         ----------------------------
                                            little |   Adj   |  Nom |
                                                  -------------------
                                                    squirrel |   N  |
                                                            ---------
```

---

## Task 4 - Shift-Recude Parser


### Part 1

Describe how the Shift-Reduce Parser works.

Shift-Reduce Parsing is a type of bottom-up parsing technique. It operates by repeatedly shifting input symbols onto a stack and then reducing them according to a grammar's production rules.

1. **Initialization**: Start with an empty stack and the input sentence to be parsed.
2. **Shift Operation**: During the shift operation, the parser reads the next input symbol and pushes it onto the stack.
3. **Reduce Operation**: During the reduce operation, the parser tries to match the symbols on the top of the stack with the right-hand side of a grammar production rule. If a match is found, the symbols are popped from the stack, and a new non-terminal symbol (corresponding to the left-hand side of the production rule) is pushed onto the stack.
4. **Repeat**: Steps 2 and 3 are repeated until the entire input sentence has been shifted onto the stack and reduced to the start symbol of the grammar. At this point, if successful, the parse is complete.


Example:

1. Initial State: <br>
Stack: [ ] <br>
Input: [the, cat, chased] <br>

2. Shift "the" onto the Stack: <br>
Stack: [the] <br>
Input: [cat, chased] <br>

3. Shift "cat" onto the Stack: <br>
Stack: [the, cat] <br>
Input: [chased] <br>

4. Reduce "Det N" to NP: <br>
Apply production rule NP -> Det N <br>
Stack: [NP] <br>
Input: [chased] <br>

5. Repeat until finished

### Part 2

Implement a Shift-Reduce Parser using NLTK (Natural Language Toolkit) to parse the given English sentence.

In [2]:
import nltk

# Sentence to parse
sentence = "the angry bear chased the frightened little squirrel".split()

# Define the grammar
grammar = nltk.CFG.fromstring("""
        S -> NP VP
        NP -> Det Nom | Det N | NP PP
        Nom -> Adj Nom | Adj N
        VP -> V Adj | V NP | V S
        PP -> P NP
        Det -> 'the' | 'a'
        N -> 'bear' | 'squirrel' 
        Adj -> 'angry' | 'frightened' | 'little'
        V -> 'saw' | 'chased'
    """)

# Shift-Reduce Parsing with NLTK
# Create a Shift-Reduce parser with the custom grammar
def build_tree(grammar, sentence):
    parser = nltk.ShiftReduceParser(grammar)
    trees = parser.parse(sentence)
    for tree in trees:
        return tree

# Parse the sentence and output the parse trees
tree = build_tree(grammar, sentence)

print("Derivation Tree:", tree)




Derivation Tree: (S
  (NP (Det the) (Nom (Adj angry) (N bear)))
  (VP
    (V chased)
    (NP
      (Det the)
      (Nom (Adj frightened) (Nom (Adj little) (N squirrel))))))


---

#### Submitting your results:

To submit your results, please:

- save this file, i.e., `ex??_assignment.ipynb`.
- if you reference any external files (e.g., images), please create a zip or rar archieve and put the notebook files and all referenced files in there.

**Remarks:**
    
- Do not copy any code from the Internet. In case you want to use publicly available code, please, add the reference to the respective code snippet.
- Check your code compiles and executes, even after you have restarted the Kernel.
- Submit your written solutions and the coding exercises within the provided spaces and not otherwise.
- Write the names of your partner and your name in the top section.